In [4]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import cgi
import shutil
import os
import json

In [5]:
# = Helper Functions =========================================================
 
def removeTags(string):
    '''
    Function to remove html tags
    '''
    return re.sub('<[^<]+?>', '', string)

In [22]:
def getBandTree(band, page):
    '''
    Function to get xml tree given the band name
    '''
    if type(page) == int:
        page = str(page)
    theURL = 'https://www.ultimate-guitar.com/search.php?band_name=' + band + \
        '&type%5B4%5D=500&rating%5B4%5D=5&approved%5B1%5D=1&page=' + page + \
        '&view_state=advanced&tab_type_group=text&app_name=ugt&order=myweight'
 
    pageBand = requests.get(theURL)
    cookies = pageBand.cookies
    return BeautifulSoup(pageBand.content), cookies

def getSongsList(band, page):
    '''
    Function to get xml tree given the band name
    '''
    
    if type(page) == int:
        page = str(page)
    theURL = 'https://www.ultimate-guitar.com/search.php?band_name=' + band + \
        '&type%5B4%5D=500&rating%5B4%5D=5&approved%5B1%5D=1&page=' + page + \
        '&view_state=advanced&tab_type_group=text&app_name=ugt&order=myweight'
 
    pageBand = requests.get(theURL)
        
    textLength = len("window.UGAPP.store.page = ")
    index = pageBand.text.find("window.UGAPP.store.page = ")
    
    jsonString = pageBand.text[index + textLength:]
    last_index = jsonString.find("window.UGAPP.store.i18n = {};")
    
#     jsonString = jsonString[:last_index].rstrip(';')
#     jsonString = jsonString[:last_index].rstrip('')
#     jsonString = jsonString[:-1]
    jsonString = jsonString[:last_index]
    
    last_index = jsonString.rfind(";")
    jsonString = jsonString[:last_index]

    return json.loads(jsonString)
 

In [23]:
def download_url(url, directory, _cookies, the_id, referer):
    """Download file from url to directory
 
    URL is expected to have a Content-Disposition header telling us what
    filename to use.
 
    Returns filename of downloaded file.
 
    """
#     response = requests.get(url, stream=True, cookies=_cookies)

    headers = {
        'authority': 'tabs.ultimate-guitar.com',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'referer': referer,
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
        'cookie': '_ga=GA1.2.1963544770.1548648296; ga_client_id=1963544770.1548648296; _ym_uid=15486482961012054356; _ym_d=1548648296; _ug_pUserHash=fecd8337-9aec-4ea1-b9cf-b7a444f18edd.1548648330; ug_sct_hidden=1; static_cache_key_v2=_ver1550524582; _ug_interstitialShown=1; _pro_abVar=2019_02_16_SITE_WIDE_BANNER_BOOST.A; _pro_abVar2=2019_02_18_PREBID_2_2.B; _csrf=C2dMf0rePFCZBc8iLLf4tXYxK7h6xPev; _gid=GA1.2.418588070.1550553843; ug_hats_group=1; official_tab_tooltip_visible=0; bbsessionhash=oJgC82F0uXgBwxs-NZ8Mo1P1HeHRXyuL; _ug_playerSideShowsPerDay=1; ugapp%3AtopBannerShow%3Axmas2018=true; _pro_landingUser=6906327291349692562; ug_react_recently_viewed=[%22forum_thread:962854%22%2C%22forum_thread:1053472%22%2C%22forum_thread:1436545%22%2C%22forum_thread:1553065%22%2C%22forum_thread:1743224%22%2C%22tab:1309073%22%2C%22forum_thread:1429292%22%2C%22tab:225442%22%2C%22tab:2143255%22%2C%22tab:225441%22]; _gat_gpptTracker=1',
    }

    params = (
        ('id', the_id),
        ('session_id', ''),
    )

    response = requests.get('https://tabs.ultimate-guitar.com/tab/download', headers=headers, params=params)
    
    if response.status_code != 200:
        raise ValueError('Failed to download')
 
    params = cgi.parse_header(
        response.headers.get('Content-Disposition', ''))[-1]
    if 'filename' not in params:
        raise ValueError('Could not find a filename')
 
    filename = re.sub('([\(\[]).*?([\)\]])', '', os.path.basename(params['filename']))
    filename = re.sub(' ','', filename)
    abs_path = os.path.join(directory, filename)
    
    with open(abs_path, 'wb') as target:
        for chunk in response.iter_content(chunk_size=128):
            target.write(chunk)
        
#         response.raw.decode_content = True
#         print(response.raw)
#         shutil.copyfileobj(response.raw, target)
 
    return filename
 
# ============================================================================
def assignAttributes(songs):
    ratings = []
    songnames = []
    types = []
    links = []
    for song in songs:
        if 'rating' in song:
            ratings.append(int(song['rating']))
        else:
            ratings.append(-1)
        
        if 'song_name' in song:
            songnames.append(song['song_name'])
        else:
            songnames.append('default')
        
        if 'marketing_type' in song:
            types.append(song['marketing_type'])
        else:
            types.append(song['type'])
            
        if 'tab_url' in song:
            links.append(song['tab_url'])
    return ratings, songnames, types, links

def main():
    band = 'coldplay'
    dfs = []    
    page = 1
    search_results = getSongsList(band, page)['data']
    while 'results' in search_results:
        songs = search_results['results']
#         mybs = bandURL.find_all('b', { "class" : "ratdig" })
#         print(mybs)
#         mybs2 = [removeTags(str(rating)) for rating in mybs]
#         songname = bandURL.find_all('a', { "class" : "link-primary _1kcZ5 _2z0cE" })
#         songname2 = [re.sub('([\(\[]).*?([\)\]])', '', removeTags(str(song)).strip()).strip() for song in songname]
#         tabType = bandURL.find_all('strong')
#         tabType2 = [removeTags(str(tab)) for tab in tabType]
        
        mybs2, songname2, tabType2, links = assignAttributes(songs)
    
        df1 = pd.DataFrame({'Rating': mybs2, 'Type': tabType2, 'Song_Name': songname2, 'Song_Links': links})
#         df2 = df1[df1.Type == 'guitar pro']
        df2 = df1[df1.Type == 'Pro']

#         df2.loc[:,'Song_Name'] = songname2
#         links = []
#         for a in songname:
#             links.append(a['href'])
#         df2.loc[:,'Song_Links'] = links
#         df3 = df2.loc[df2.groupby(['Song_Name'], sort=False)['Rating'].idxmax()]


#         df3 = df2.loc[df2['Rating'].idxmax()]
#         dfs.append(df3)
        dfs.append(df2)
    
        page += 1
        search_results = getSongsList(band, page)['data']
    
    tot_df = pd.concat(dfs)
    song_links_list = list(tot_df.Song_Links)
    for i in range(len(tot_df)):
        print(song_links_list[i])
        webPage = requests.get(song_links_list[i])
        
#         soup = BeautifulSoup(webPage.content)
#         print(soup)
#         tab_id = soup.find_all("input", {"type" : "hidden", "name" : "id", "id" : "tab_id"})
#         print(tab_id)
#         the_id = tab_id[0].get('value')
        
        the_id = song_links_list[i][song_links_list[i].rfind('_') + 1:]
        
        mydir = os.path.dirname('out/' +band + '/')
        if not os.path.exists(mydir):
            os.makedirs(mydir)
        download_url('https://tabs.ultimate-guitar.com/tab/download?id='+str(the_id), mydir, webPage.cookies, str(the_id), song_links_list[i])
        

In [24]:
main()

https://tabs.ultimate-guitar.com/tab/coldplay/yellow_guitar_pro_216843
https://tabs.ultimate-guitar.com/tab/coldplay/yellow_guitar_pro_733038
https://tabs.ultimate-guitar.com/tab/coldplay/yellow_guitar_pro_1150280
https://tabs.ultimate-guitar.com/tab/coldplay/the_scientist_guitar_pro_803055
https://tabs.ultimate-guitar.com/tab/coldplay/the_scientist_guitar_pro_2040437
https://tabs.ultimate-guitar.com/tab/coldplay/the_scientist_guitar_pro_2085925
https://tabs.ultimate-guitar.com/tab/coldplay/fix_you_guitar_pro_399086
https://tabs.ultimate-guitar.com/tab/coldplay/fix_you_guitar_pro_488757
https://tabs.ultimate-guitar.com/tab/coldplay/viva_la_vida_guitar_pro_2024250
https://tabs.ultimate-guitar.com/tab/coldplay/viva_la_vida_guitar_pro_1455911
https://tabs.ultimate-guitar.com/tab/coldplay/viva_la_vida_guitar_pro_1678923
https://tabs.ultimate-guitar.com/tab/coldplay/clocks_guitar_pro_1088190
https://tabs.ultimate-guitar.com/tab/coldplay/everglow_guitar_pro_1808803
https://tabs.ultimate-guit

In [36]:
import guitarpro
from os import listdir
 
# = Helper functions =========================================================
 
def unfold_tracknumber(tracknumber, tracks):
    """Substitute '*' with all track numbers except for percussion tracks."""
    if tracknumber == '*':
        for number, track in enumerate(tracks, start=1):
            if not track.isPercussionTrack:
                yield number
    else:
        yield tracknumber
 
 
def transpose(myfile, track):
    '''
    Get pertinent information from guitar pro files and write to text file
    '''
    myfile.write("%s" % 'strgs: ' +' '.join([str(string) for string in track.strings]) + ' ')
    myfile.write("%s" % 'fc: ' +str(track.fretCount) + ' ')
    measure1 = track.measures[0]
    myfile.write("%s" % str(measure1.keySignature) + ' ')
    myfile.write("%s" % 'len:' + str(measure1.length) + ' ')
    myfile.write("%s" % 'tmpo:' + str(measure1.tempo) + '\n')
    i = 1
    for measure in track.measures:
        myfile.write("%s" % 'Num:' + str(i) + ' ')
        myfile.write("%s" % 'mst:' + str(measure.start) + '\n')
        for voice in measure.voices:
            for beat in voice.beats:
                myfile.write("%s" % 'vst:' + str(beat.start) + '\n')
                for note in beat.notes:
                    myfile.write("%s" % 'S:' + str(note.string) + ' ')
                    myfile.write("%s" % 'V:' + str(note.value) + '\n')
        i += 1
 
# =================================================================
 
def main():
 
    band = 'coldplay'
    mydir = 'out/' + band
    files = listdir(mydir)
    myfile = open('allTabs.txt', 'w')
    for gpfile in files:
        if gpfile[-1] not in ['3', '4', '5']:
            continue
        print(gpfile)
        curl = guitarpro.parse(mydir + '/'+ gpfile)
        print(len(curl.tracks))
        transpose(myfile, curl.tracks[0])
        myfile.write('\n\r\n\r')
    myfile.close()
 

In [37]:
main()

Coldplay-LoversInJapan.gp5
9
Coldplay-TheScientist.gp3
7
Coldplay-FixYou.gp5
9
Coldplay-FixYou.gp4
9
Coldplay-ARushOfBloodToTheHead.gp3
4
Coldplay-DontPanic.gp5
10
Coldplay-HowYouSeeTheWorldNo2.gp5
8
Coldplay-Yellow.gp5
7
Coldplay-Lost.gp5
11
Coldplay-VioletHill.gp5
8
Coldplay-CrestsOfWaves.gp5
7
Coldplay-DontPanic.gp3
11
Coldplay-Yellow.gp3
4
Coldplay-OdeToDeodorant.gp5
8
Coldplay-136.gp5
5
Coldplay-LifeIsForLiving.gp5


KeyboardInterrupt: 